# Notebook Setup

In [16]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"

In [17]:
if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Import Modules

In [18]:
# install required modules quietly
required_packages = ['geopandas', 'pyspark', 'azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.1 MB/s eta 0:00:00


In [19]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import math
import json

import pyspark.pandas as pd
from calendar import monthrange
from datetime import datetime
from io import BytesIO

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
if IN_COLLAB:
  os.chdir(MY_HOME_ABS_PATH)
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from edahelpers import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [20]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

## Define Local Files System Constants

In [21]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = tmp_dir
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/CO2_flux_gpp_modeling/DS_capstone_23Spring_CO2/Data/half_hourly_data"

site_metadata_filename = data_dir + os.sep + 'site-metadata.csv'

# Output
tag = "1_raw"
container = "gold-samples-data"
blob_name = f"gold_samples_trim_v_{tag}.parquet" #Advisor suggested features only

In [22]:
# "Golden" Sites
tier1_sites = ["US-MMS", "US-Vcp", "FR-Pue", "CH-Lae", "US-Var", "US-Ne2", "ES-LJu", "US-Ton"]
tier2_sites = ["US-UMB", "US-Me2", "FI-Hyy", "US-NR1", "IT-Lav", "US-Wkg", "US-ARM", "US-SRM"]

target_sites = tier1_sites + tier2_sites

# Get Gold Sample Site Data

In [23]:
# Load site metadata
included_site_features = ['site_id', 'filename', 'elevation', 'lat', 'long',
                          'koppen_sub', 'koppen_main', 'koppen_name', 'koppen_main_name',
                          'c3c4', 'c4_percent']
site_metadata_df = pd.read_csv(site_metadata_filename, usecols = included_site_features)

# only focus on target sites
site_metadata_df= site_metadata_df.loc[site_metadata_df['site_id'].isin(target_sites)]
print(f"size:{site_metadata_df.shape}")
site_metadata_df.reset_index(inplace=True, drop=True)
site_metadata_df

size:(16, 11)


,site_id,elevation,lat,long,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent,filename
0,FR-Pue,270.00000,43.74130,3.59570,8,3,Csa,Temperate,C3,6.59000,data_full_half_hourly_raw_v0_1_FR-Pue.csv
1,US-NR1,3050.00000,40.03290,-105.54640,27,4,Dfc,Cold,C3,0.35000,data_full_half_hourly_raw_v0_1_US-NR1.csv
2,US-Ne2,362.00000,41.16487,-96.47010,25,4,Dfa,Cold,rotation,48.91000,NaN
3,US-SRM,1120.00000,31.82140,-110.86610,6,2,BSh,Arid,C3,55.39000,data_full_half_hourly_raw_v0_1_US-SRM.csv
4,US-Ton,177.00000,38.43160,-120.96598,8,3,Csa,Temperate,C3,0.00000,data_full_half_hourly_raw_v0_1_US-Ton.csv
5,US-Var,129.00000,38.41330,-120.95070,8,3,Csa,Temperate,C3,0.00000,data_full_half_hourly_raw_v0_1_US-Var.csv
6,US-Wkg,1531.00000,31.73650,-109.94190,7,2,BSk,Arid,C4,40.93000,data_full_half_hourly_raw_v0_1_US-Wkg.csv
7,US-ARM,314.00000,36.60580,-97.48880,14,3,Cfa,Temperate,mix,15.97000,data_full_half_hourly_raw_v0_1_US-ARM.csv
8,US-MMS,275.00000,39.32320,-86.41310,25,4,Dfa,Cold,C3,42.28000,NaN
9,US-Me2,1253.00000,44.45230,-121.55740,18,4,Dsb,Cold,C3,0.03000,data_full_half_hourly_raw_v0_1_US-Me2.csv


# Run Data Pipeline on Gold Sample Sites

In [29]:
all_features = ['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen']

In [30]:
def data_cleanup(site_id_file_df, target, target_qc, features):

    data_df = None
    # qc_flag_dtype = CategoricalDtype([0, 1, 2, 3], ordered=True)
    qc_flags_features = [s for s in included_features if "_QC" in s]

    # Iterate through each site:
    for i, r in site_id_file_df.iterrows():        
        if not r.filename or type(r.filename) != type(""):
            print(f'\nERROR: {r.site_id} is mssing hourly data.')
            continue

        # Get only `included_features` from file
        local_filename = raw_data_dir + os.sep + r.filename
        site_df = pd.read_csv(local_filename, usecols = included_features +  target_variable + target_variable_qc)
        site_df['datetime'] = pd.to_datetime(site_df['datetime'])
        site_df['date'] = pd.to_datetime(site_df['date'])
        site_df['minute'] = site_df['datetime'].dt.minute
        if len(qc_flags_features) != 0:
            site_df[qc_flags_features] = site_df[qc_flags_features].astype('int')
        site_df['site_id'] = r.site_id

        # Remove zero or negative SW
        site_df.drop(site_df[site_df['SW_IN_ERA'] <= 0].index, inplace = True)

        # Drop rows with NAs for Target Variable
        site_df.dropna(subset=target_variable, axis=0, inplace=True)

        # Drop rows with bad NEE_VUT_REF_QC (aka bad GPP records)
        site_df.drop(site_df[site_df[target_variable_qc[0]] == 3].index, inplace = True)
        site_df.drop(target_variable_qc, axis=1, inplace=True)

        # Drop rows with any NA
        site_df.dropna(axis=0, inplace=True)

        print(f"{r.site_id}: {site_df.shape}")
        if type(data_df) == type(None):
            data_df = site_df
        else:
            data_df = pd.concat([data_df, site_df])
            
    return data_df

def merg_site_metadata(data_df, site_metadata_df):
    # Merge with Site Metadata
    data_df = data_df.merge(site_metadata_df, how='left', left_on='site_id', right_on='site_id')
    return data_df

In [31]:
included_features = ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA',
                     'datetime', 'year', 'month', 'day', 'hour', 'date',
                     'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 
                     'IGBP', 'koppen']
target_variable_qc = ['NEE_VUT_REF_QC']
target_variable = ['GPP_NT_VUT_REF']

# Get Train Dataset
data_df = data_cleanup(site_metadata_df[['site_id','filename']],
                  target_variable, target_variable_qc,
                  included_features)
print(f"Data size after cleanup: {data_df.shape}")

data_df = merg_site_metadata(data_df, site_metadata_df.drop(['filename'], axis=1))
print(f"Data size after after merged with site metadata: {data_df.shape}")
display(data_df.head())


FR-Pue: (117200, 27)
US-NR1: (98652, 27)

ERROR: US-Ne2 is mssing hourly data.
US-SRM: (95419, 27)
US-Ton: (113031, 27)
US-Var: (119950, 27)
US-Wkg: (93319, 27)
US-ARM: (125756, 27)

ERROR: US-MMS is mssing hourly data.
US-Me2: (99780, 27)
US-UMB: (70639, 27)
US-Vcp: (78491, 27)
CH-Lae: (112718, 27)
ES-LJu: (112724, 27)
FI-Hyy: (127362, 27)
IT-Lav: (120885, 27)
Data size after cleanup: (1485926, 27)
Data size after after merged with site metadata: (1485926, 36)


,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,GPP_NT_VUT_REF,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent
0,5.31100,25.01600,272.21800,1.70800,0.00000,97.93900,-0.53574,2001-01-01 08:30:00,2001,1,1,8,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,3,Csa,Temperate,C3,6.59000
1,5.74400,59.73400,272.21800,1.73800,0.00000,97.93900,0.86438,2001-01-01 09:00:00,2001,1,1,9,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,270.00000,43.74130,3.59570,8,3,Csa,Temperate,C3,6.59000
2,6.17600,91.23500,272.21800,1.76700,0.00000,97.93900,-0.02627,2001-01-01 09:30:00,2001,1,1,9,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,3,Csa,Temperate,C3,6.59000
3,6.60800,79.26400,333.93300,1.79700,0.05000,97.93900,-0.17229,2001-01-01 10:00:00,2001,1,1,10,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,270.00000,43.74130,3.59570,8,3,Csa,Temperate,C3,6.59000
4,7.04300,94.92900,333.93300,1.81700,0.00000,97.92300,1.20865,2001-01-01 10:30:00,2001,1,1,10,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,3,Csa,Temperate,C3,6.59000


In [32]:
data_df.describe()

,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,GPP_NT_VUT_REF,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,minute,elevation,lat,long,koppen_sub,koppen_main,c4_percent
count,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000
mean,14.87724,377.56323,316.15936,10.11574,0.03757,90.79849,5.86211,2009.75172,6.53593,15.74438,11.89376,0.29739,0.53815,0.13063,0.07894,0.24123,0.04288,0.07109,0.25797,0.20036,0.11929,14.99925,971.07777,41.75447,-62.35813,16.51431,3.34981,8.82611
std,9.04516,268.53741,47.26856,9.61817,0.18091,8.84375,7.52193,4.52259,2.98991,8.81076,3.77390,0.13132,0.21932,0.07137,0.05540,0.06086,0.03907,0.04160,0.06960,0.09369,0.07861,15.00001,866.88847,7.77408,59.15444,8.71496,0.69389,15.99510
min,-29.74000,0.00100,142.77000,0.00000,0.00000,67.40500,-49.73720,2001.00000,1.00000,1.00000,3.00000,-0.11958,-0.18252,-0.01715,0.00540,0.03050,0.00000,0.00000,0.00000,0.01320,0.00000,0.00000,129.00000,31.73650,-121.55740,6.00000,2.00000,0.00000
25%,8.69200,138.70625,283.86900,3.18000,0.00000,85.24200,0.45157,2006.00000,4.00000,8.00000,9.00000,0.20336,0.34150,0.08147,0.03370,0.19840,0.01960,0.04220,0.20210,0.11890,0.05150,0.00000,234.00000,36.60580,-110.86610,8.00000,3.00000,0.00000
50%,14.93100,344.52850,317.35000,6.98700,0.00000,93.16700,3.23428,2010.00000,7.00000,16.00000,12.00000,0.28756,0.56813,0.11822,0.06390,0.23080,0.03400,0.06320,0.26110,0.18680,0.10330,0.00000,689.00000,40.03290,-97.48880,14.00000,3.00000,0.04000
75%,21.28200,587.46300,349.28900,13.73200,0.00000,98.71700,9.48720,2013.00000,9.00000,23.00000,15.00000,0.36012,0.70597,0.15872,0.11660,0.27750,0.05450,0.09120,0.31520,0.27910,0.18240,30.00000,1531.00000,45.55980,3.59570,26.00000,4.00000,10.72000
max,42.58700,1094.34100,473.01100,75.68400,15.49300,103.38300,85.03090,2020.00000,12.00000,31.00000,23.00000,2.38835,0.93551,0.42385,0.79710,0.77290,0.76890,0.78650,0.46660,0.42800,0.35730,30.00000,3050.00000,61.84741,24.29477,27.00000,4.00000,55.39000


In [33]:
data_df.site_id.unique()

array(['FR-Pue', 'US-NR1', 'US-SRM', 'US-Ton', 'US-Var', 'US-Wkg',
       'US-ARM', 'US-Me2', 'US-UMB', 'US-Vcp', 'CH-Lae', 'ES-LJu',
       'FI-Hyy', 'IT-Lav'], dtype=object)

# Upload Data to Azure Storage Blob as Parquet
**Run with Caution!!!**

In [34]:
# Upload to Azure Storage Blob
# ref: https://stackoverflow.com/a/54666079
parquet_file = BytesIO()
data_df.to_parquet(parquet_file, engine='pyarrow')
parquet_file.seek(0)

azStorageClient = AzStorageClient(az_cred_file)
azStorageClient.uploadBlob(container, blob_name, parquet_file, overwrite=True)

File uploaded to gold-samples-data/gold_samples_trim_v_1_raw.parquet


## Should be no NA data

In [35]:
total_record_count = data_df.shape[0]
na_df = pd.DataFrame(data_df.isna().sum())
na_df["percentage"] = (na_df / total_record_count)
na_df.rename(columns={0:"count"}, inplace=True)

In [36]:
na_df.loc[(na_df['count'] != 0)].sort_values("percentage", ascending=False)

,count,percentage
